In [12]:
# Process the entire "Winnie-the-Pooh" (A. A. Milne) novel using the Document AI batch processing API.
input_file_name   = "sample-batch-ocr.pdf"
output_file_name  = "cepf_batch_ocr.txt"

bucket_name = "qwiklabs-gcp-01-452f5b09addd-cepf-documentai"  # Don't use the leading gs://
gcs_input_uri  = "gs://" + bucket_name + "/" + input_file_name
gcs_output_uri = "gs://" + bucket_name + "/" + output_file_name

print(gcs_input_uri)
print(gcs_output_uri)

project_id   = "qwiklabs-gcp-01-452f5b09addd"
location     = "us"
processor_id = "71f15e92ff1df451"

name = f"projects/{project_id}/locations/{location}/processors/{processor_id}"

gs://qwiklabs-gcp-01-452f5b09addd-cepf-documentai/sample-batch-ocr.pdf
gs://qwiklabs-gcp-01-452f5b09addd-cepf-documentai/cepf_batch_ocr.txt


In [16]:
from google.api_core.client_options import ClientOptions
import google.cloud.documentai as docai

api_endpoint=location + "-documentai.googleapis.com"
mime_type="application/pdf"

client_options = ClientOptions(api_endpoint=api_endpoint)
client = docai.DocumentProcessorServiceClient(client_options=client_options)
#gcs_source = docai.GcsSource(uri=uri2input_file)
gcs_document = docai.GcsDocument(gcs_uri=uri2input_file, mime_type=mime_type)
gcs_documents= docai.GcsDocuments(documents=[gcs_document])
input_config = docai.BatchDocumentsInputConfig(gcs_documents=gcs_documents)

field_mask = "text" # field_mask = "text,entities,pages.pageNumber"
gcs_output_config = docai.DocumentOutputConfig.GcsOutputConfig(gcs_uri=gcs_output_uri, field_mask=field_mask)

In [ ]:
output_config = docai.DocumentOutputConfig(gcs_output_config=gcs_output_config)

# BatchProcess returns a Long Running Operation (LRO)
operation = client.batch_process_documents(request)

# Continually polls the operation until it is complete.
# This could take some time for larger files
# Format: projects/{project_id}/locations/{location}/operations/{operation_id}
try:
    print(f"Waiting for operation {operation.operation.name} to complete...")
    operation.result(timeout=timeout)
# Catch exception when operation doesn't finish before timeout
except (RetryError, InternalServerError) as e:
    print(e.message)

In [ ]:
# Load the document as bytes
with open( input_file_name, 'rb' ) as image:
    image_content = image.read()

In [4]:
result = client.process_document(request=request)
document = result.document
# print(document.text)

InvalidArgument: 400 Document pages exceed the limit: 15 got 181 [reason: "PAGE_LIMIT_EXCEEDED"
domain: "documentai.googleapis.com"
metadata {
  key: "page_limit"
  value: "15"
}
metadata {
  key: "pages"
  value: "181"
}
]